In [1]:
from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer, AutoTokenizer
import torch

from lava import LavaModel
model = LavaModel.from_lava_pretrained('roberta-base', 'facebook/bart-base')

from utils.datasets_config import get_dataset

dset = get_dataset('gigaword', 'facebook/bart-base')
print(dset)

tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')




Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You passed along `num_labels=3` with an incompatible id to label map: {0: 'LABEL_0', 1: 'LABEL_1'}. The number of labels wil be overwritten to 2.
Found cached dataset gigaword (/home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6/cache-6816d5b30b6eeb9a.arrow
Loading cached processed dataset at /home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6/cache-6aeaeef7bff602c5.arrow
Loading cached processed dataset at /home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6/cache-29041409e3286f61.arrow


Number of gigaword training examples: 3803957
Number of gigaword validation examples: 189651
Number of gigaword testing examples: 1951
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'input_texts', 'label_texts'],
        num_rows: 3803957
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'input_texts', 'label_texts'],
        num_rows: 189651
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'input_texts', 'label_texts'],
        num_rows: 1951
    })
})


In [2]:
ckpt = torch.load('lava-ce/checkpoint-350000/pytorch_model.bin')

In [78]:
model.load_state_dict(ckpt)

<All keys matched successfully>

In [4]:
d = next(iter(dset['train'].shard(300, 1).shuffle(0)))
decoder_input_ids = (d['labels'] == 1).long()



Loading cached shuffled indices for dataset at /home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6/cache-b3cb70f2e5b25a48.arrow


In [79]:
o = model(
    input_ids = d['input_ids'].to(model.device).unsqueeze(0),
    attention_mask = d['attention_mask'].to(model.device).unsqueeze(0),
#     labels = d['labels'].to(model.device).unsqueeze(0),
)

ValueError: too many values to unpack (expected 2)

In [21]:
d

{'input_ids': tensor([[    0,   179,    10, 50264,  8840,    59,     5,  1737,     8,   806,
          1437, 17363,    32,   634, 19864,  2793,  4247,     7, 50264,    11,
            23,    10,   592, 11745, 48091,     7,   489, 50264,  1334,  3711,
            30,  2147,    31, 50264,    66,  1437,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [84]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained('roberta-base')
# model = LavaModel.from_lava_pretrained('roberta-base', 'facebook/bart-base')


In [86]:
d = tokenizer("in a city passionate about the environment and technology <mask> commuters are <mask> smart phones to check in at a social networkingSCP to keep ater threatened by climate from checking out <mask> <mask><mask><mask><mask><mask><mask> <mask> and so on.", return_tensors = 'pt')

o = model(
    input_ids = d['input_ids'].to(model.device),
    attention_mask = d['attention_mask'].to(model.device),
)

In [87]:
d

{'input_ids': tensor([[    0,   179,    10,   343,  8840,    59,     5,  1737,     8,   806,
         50264, 17363,    32, 50264,  2793,  4247,     7,  1649,    11,    23,
            10,   592, 11745, 48091,     7,   489,    10,  1334,  3711,    30,
          2147,    31,  8405,    66, 50264, 50264, 50264, 50264, 50264, 50264,
         50264, 50264,     8,    98,    15,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [88]:
o.logits.argmax(2)

tensor([[    0,   179,    10,   343,  8840,    59,     5,  1737,     8,   806,
             6, 17363,    32,   634,  2793,  4247,     7,  1649,    11,    23,
            10,   592, 11745,  1082,     7,   489,    10,  1334,  3711,    30,
          2147,    31,  8405,    66,    49,     6,     6,     6,     6,     6,
             6,     6,     8,    98,    15,     4,     2]])

In [89]:
tokenizer.decode(o.logits.argmax(2).tolist()[0], skip_special_tokens=False)


'<s>in a city passionate about the environment and technology, commuters are using smart phones to check in at a social networking site to keep ater threatened by climate from checking out their,,,,,,, and so on.</s>'

In [83]:
tokenizer.decode(o.logits.argmax(2).tolist()[0], skip_special_tokens=False)


'<s>in a city passionate about the environment and technology   are using smart phones to check in at a social networking meeting to keep a person threatened by climate from checking out         and so on </s>'

In [58]:
print(tokenizer.decode(d['input_ids'][0], skip_special_tokens=True))
print()
# print(tokenizer.decode(d['labels'][0], skip_special_tokens=True))

in a city passionate about the environment and technology commuters are smart phones to check in at a social networkingSCP to keep ater threatened by climate from checking out



In [39]:
scores = model.decoder(input_ids = d['input_ids'].to(model.device).unsqueeze(0), decoder_input_ids = 50264 * torch.ones_like(d['input_ids']).to(model.device).unsqueeze(0)).end_logits
scores

ValueError: too many values to unpack (expected 2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(scores.tolist()[0])

In [ ]:
model.decoder(input_ids = d['input_ids'].to(model.device).unsqueeze(0), decoder_input_ids = 50264 * torch.ones_like(d['labels']).to(model.device).unsqueeze(0)).end_logits